In [1]:
# importing dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# Retrieve the parent divs for all articles
title_results = soup.find_all('div', class_='content_title')
paragraph_results = soup.find_all('div', class_='article_teaser_body')

print(paragraph_results)
# loop over results to get article data
# for result in paragraph_results:
#     # scrape the article title 
#     title = result.find('a').text
    
    # scrape the article paragraph
#     paragraph = result.find('div', class_='article_teaser_body')
    
#     # print article data
#     print('-----------------')
#     print(title)
#     print(paragraph)

[]
